In [9]:
# 1. 获得 URL - 从 Seeking Alpha 获取所有 Earnings Call Transcript 链接

import time
import json
import random
import os
import re
from datetime import datetime
from pathlib import Path

import asyncio
from playwright.async_api import async_playwright
import nest_asyncio
nest_asyncio.apply()

# ========== 配置区 ==========
# 14 家公司列表
TICKER_LIST = [
    "JPM",  # JPMorgan Chase
    "BK",   # Bank of New York Mellon
    "DAL",  # Delta Airlines
    "C",    # Citigroup
    "WFC",  # Wells Fargo
    "BAC",  # Bank of America
    "TSM",  # TSMC
    "MS",   # Morgan Stanley
    "GS",   # Goldman Sachs
    "BLK",  # BlackRock
    "MTB",  # M&T Bank
    "STT",  # State Street
    "PNC",  # PNC Financial
    "BR",   # Broadridge Financial
    # 还需要补齐第15、16家（根据当前 earnings calendar）
]

# 只获取近10年的数据（2015年及以后）
MIN_YEAR = 2023

PROJECT_ROOT = Path("..").resolve()
INDEX_DIR = PROJECT_ROOT / "data" / "transcript_index"
INDEX_DIR.mkdir(parents=True, exist_ok=True)

print(f"项目根目录: {PROJECT_ROOT}")
print(f"URL索引保存目录: {INDEX_DIR}")
print(f"目标公司数: {len(TICKER_LIST)}")
print(f"时间范围: {MIN_YEAR}年及以后（近10年）")

项目根目录: /Users/xinyuewang/Desktop/1.27
URL索引保存目录: /Users/xinyuewang/Desktop/1.27/data/transcript_index
目标公司数: 14
时间范围: 2023年及以后（近10年）


In [10]:
#人工验证
async def wait_for_captcha_and_resume(page):
    """
    检测验证码页面，如果存在，则暂停程序等待用户手动解决。
    """
    print("正在检查是否存在验证码...")
    try:
        captcha_button = page.get_by_text("按住").first
        await captcha_button.wait_for(state="visible", timeout=5000)

        print("\n" + "="*50)
        print("检测到验证码！请在浏览器中手动完成验证。")
        print("完成后，请回到此终端并按 Enter 键继续。")
        print("="*50 + "\n")

        input("按下 Enter 键以继续...")

        await page.wait_for_load_state('domcontentloaded')
        print("验证码已解决，程序继续。")

    except Exception:
        print("未检测到验证码，继续执行。")
        pass

In [11]:
def get_completed_tickers(progress_file="progress.txt"):
    if os.path.exists(progress_file):
        with open(progress_file, "r") as f:
            return set(line.strip() for line in f)
    return set()

def mark_as_completed(ticker, progress_file="progress.txt"):
    with open(progress_file, "a") as f:
        f.write(f"{ticker}\n")

def mark_as_excluded(ticker, exclude_file="exclude_transcript.txt"):
    if os.path.exists(exclude_file):
        with open(exclude_file, "a") as f:
            f.write(f"{ticker}\n")
    else:
        with open(exclude_file, "w") as f:
            f.write(f"{ticker}\n")

def mark_as_failed(ticker, failed_file="failed.txt"):
    if os.path.exists(failed_file):
        with open(failed_file, "a") as f:
            f.write(f"{ticker}\n")
    else:
        with open(failed_file, "w") as f:
            f.write(f"{ticker}\n")

In [ ]:
def extract_year_from_title(title: str) -> int:
    """从标题中提取年份（例如 "Q1 2024 Earnings" -> 2024）"""
    # 匹配 4 位数字年份
    match = re.search(r'\b(20\d{2})\b', title)
    if match:
        return int(match.group(1))
    return None


def check_missing_years(existing_results: list, min_year: int = MIN_YEAR) -> set:
    """检查已存在的 results 中缺失哪些年份（2015年及以后）"""
    years_found = set()
    for item in existing_results:
        title = item.get('title', '')
        year = extract_year_from_title(title)
        if year and year >= min_year:
            years_found.add(year)
    
    # 期望的年份范围（当前年份往前推10年）
    current_year = datetime.now().year
    expected_years = set(range(min_year, current_year + 1))
    
    missing_years = expected_years - years_found
    return missing_years


async def connect_and_scrape(ticker_list, min_year: int = MIN_YEAR, check_missing: bool = True):
    """
    连接到 Chrome 并抓取所有 ticker 的 URL
    
    Args:
        ticker_list: 要处理的 ticker 列表
        min_year: 最小年份（只获取此年份及以后的数据）
        check_missing: 是否检查缺失年份并重新抓取
    """
    print("🚀 正在连接 Chrome（端口 9222）...")

    async with async_playwright() as p:
        # 连接到你已经用 9222 开启的 Chrome
        try:
            browser = await p.chromium.connect_over_cdp("http://localhost:9222")
            context = browser.contexts[0] if browser.contexts else await browser.new_context()
            print("✅ 成功连接到 Chrome")
        except Exception as e:
            print(f"❌ 连接失败: {e}")
            print("请确保已运行: nohup /Applications/Google\\ Chrome.app/Contents/MacOS/Google\\ Chrome --remote-debugging-port=9222 --user-data-dir=/tmp/chrome_sa >/tmp/chrome_sa.log 2>&1 &")
            return

        # 进度文件读取
        completed_tickers = get_completed_tickers()  # 默认 progress.txt（看你函数默认值）
        excluded_tickers = get_completed_tickers(progress_file="exclude_transcript.txt")
        failed_tickers = get_completed_tickers(progress_file="failed.txt")

        all_processed = completed_tickers.union(excluded_tickers).union(failed_tickers)
        tickers_to_process = [t for t in ticker_list if t not in all_processed]

        if not tickers_to_process:
            print("🎉 所有 tickers 已完成/已排除/已失败，无需再次运行。")
            await browser.close()
            return

        print(f"📄 将从 {len(tickers_to_process)} 个未完成的 tickers 开始爬取...")

        # 输出目录
        INDEX_DIR.mkdir(parents=True, exist_ok=True)

        # 确保记录文件存在
        for fp in ["exclude_transcript.txt", "failed.txt"]:
            if not os.path.exists(fp):
                with open(fp, "w", encoding="utf-8") as _:
                    pass

        # 逐个 ticker 处理
        for ticker in tickers_to_process:
            base_url = f"https://seekingalpha.com/symbol/{ticker}/earnings/transcripts"
            
            # 检查是否已有 JSON 文件
            json_path = INDEX_DIR / f"{ticker}.json"
            existing_results = []
            if json_path.exists() and json_path.stat().st_size > 0:
                try:
                    with open(json_path, 'r', encoding='utf-8') as f:
                        existing_results = json.load(f)
                    print(f"📋 {ticker}: 已存在 {len(existing_results)} 条 URL")
                    
                    # 检查缺失年份
                    if check_missing:
                        missing_years = check_missing_years(existing_results, min_year)
                        if missing_years:
                            print(f"⚠️ {ticker}: 缺失年份 {sorted(missing_years)}，将重新抓取")
                        else:
                            print(f"✅ {ticker}: 所有年份（{min_year}年及以后）都已存在，跳过")
                            mark_as_completed(ticker)
                            continue
                except json.JSONDecodeError:
                    print(f"⚠️ {ticker}: JSON 文件损坏，将重新抓取")
                    existing_results = []

            try:
                # 复用已有 seekingalpha 标签页，否则新开
                page = None
                for existing_page in context.pages:
                    if "seekingalpha.com" in (existing_page.url or ""):
                        page = existing_page
                        break
                if page is None:
                    page = await context.new_page()

                print(f"\n➡️ 正在处理 {ticker}...")
                print(f"➡️ 导航到 {base_url}")
                await page.goto(base_url, timeout=60000)
                await wait_for_captcha_and_resume(page)

                # 检查页面是否有 transcripts（有时该 ticker 没内容）
                print("🔍 检查页面是否包含 transcripts 列表...")
                transcripts_button = await page.query_selector('a[data-test-id="earnings/transcripts"]')
                if not transcripts_button:
                    print(f"⚠️ {ticker} 页面不包含 transcripts 区域，写空 JSON 并标记排除。")

                    file_path = os.path.join(INDEX_DIR, "BM.json")
                    with open(file_path, "w", encoding="utf-8") as f:
                        json.dump([], f, indent=2, ensure_ascii=False)

                    mark_as_excluded(ticker)
                    print(f"✅ 已保存空文件: {file_path}")
                    continue

                # ✅ 分页抓取：只保留 Earnings Call Transcript
                print("📄 使用分页模式抓取所有页面（只保留 Earnings Call Transcript）...")

                results = []
                seen_links = set()
                page_num = 1

                while True:
                    # 限制：最多抓取10页，避免无限循环
                    if page_num > 10:
                        print(f"🛑 已达到最大页数限制（10页），停止翻页。")
                        break
                    paged_url = f"{base_url}?page={page_num}"
                    print(f"➡️ 抓取第 {page_num} 页: {paged_url}")

                    await page.goto(paged_url, timeout=60000)
                    await wait_for_captcha_and_resume(page)

                    # 如果这一页连列表都等不到，说明没内容/到头了
                    try:
                        await page.wait_for_selector(
                            'h3 a[data-test-id="post-list-item-title"]',
                            timeout=15000
                        )
                    except Exception:
                        print(f"🛑 第 {page_num} 页没有检测到文章列表，停止翻页。")
                        break

                    article_links = await page.query_selector_all(
                        'h3 a[data-test-id="post-list-item-title"]'
                    )
                    if not article_links:
                        print(f"🛑 第 {page_num} 页链接数为 0，停止翻页。")
                        break
                    # ✅ 新增：检查“无内容提示”
                    no_content = await page.query_selector(
                        "text=There are currently no headlines available"
                    )
                    if no_content:
                        print(f"🛑 第 {page_num} 页显示无内容提示，停止翻页。")
                        break

                    if not article_links:
                        print(f"🛑 第 {page_num} 页 article_links 为空，停止翻页。")
                        break

                    print(f"📄 第 {page_num} 页找到 {len(article_links)} 个标题链接（未过滤）")

                    page_added = 0
                    for link_elem in article_links:
                        title = (await link_elem.inner_text() or "").strip()
                        href = await link_elem.get_attribute("href")
                        if not href:
                            continue

                        full_link = f"https://seekingalpha.com{href.split('#')[0]}"

                        # ✅ 过滤：只保留季度 Earnings Call Transcript
                        if "Earnings Call Transcript" not in title:
                            continue

                        # ✅ 过滤：只保留当前 ticker 的电话（避免 MS 页面混入 EV 等）
                        # 规则：标题里必须包含 "(TICKER)"，例如 "Morgan Stanley (MS) ..."
                        ticker_tag = f"({ticker})"
                        if ticker_tag not in title:
                            continue
                        
                        # ✅ 过滤：只保留指定年份及以后的数据
                        year = extract_year_from_title(title)
                        if year is None or year < min_year:
                            continue

                        # 去重（检查链接和标题）
                        if full_link in seen_links:
                            continue
                        seen_links.add(full_link)
                        
                        # 如果已有结果，检查是否重复
                        if any(item.get('link') == full_link for item in existing_results):
                            continue

                        results.append({"title": title, "link": full_link})
                        page_added += 1

                    print(f"✅ 第 {page_num} 页过滤后新增 {page_added} 条 Earnings Call Transcript")

                    # 可选：如果从第2页开始连续没有新增，通常后面也不会有了
                    # if page_added == 0 and page_num >= 2:
                    #     print("🛑 连续页未发现 Earnings Call Transcript，提前停止翻页。")
                    #     break

                    page_num += 1
                    await asyncio.sleep(random.uniform(1.0, 2.0))

                # 合并新旧结果
                if existing_results:
                    # 合并并去重
                    existing_links = {item.get('link') for item in existing_results}
                    new_results = [r for r in results if r.get('link') not in existing_links]
                    all_results = existing_results + new_results
                    print(f"📄 {ticker} 本次新增 {len(new_results)} 条，总计 {len(all_results)} 条")
                else:
                    all_results = results
                    print(f"📄 {ticker} 最终共收集到 {len(results)} 条 Earnings Call Transcript（去重后）")
                
                # 再次过滤：只保留当前 ticker 的记录，清理掉历史 JSON 中混入的其他公司（例如 EV）
                ticker_tag = f"({ticker})"
                all_results = [item for item in all_results if ticker_tag in item.get('title', '')]

                # 按年份统计
                years_found = {}
                for item in all_results:
                    year = extract_year_from_title(item.get('title', ''))
                    if year:
                        years_found[year] = years_found.get(year, 0) + 1
                print(f"📊 {ticker} 年份分布: {dict(sorted(years_found.items()))}")

                # 保存结果
                json_path = INDEX_DIR / f"{ticker}.json"
                with open(json_path, "w", encoding="utf-8") as f:
                    json.dump(all_results, f, indent=2, ensure_ascii=False)

                mark_as_completed(ticker)
                print(f"✅ {ticker} 数据已成功保存至 {json_path}")

            except Exception as e:
                print(f"❌ 抓取 {ticker} 失败: {e}")
                mark_as_failed(ticker)

        await browser.close()
        print("✅ 所有任务完成，浏览器已关闭。")

In [13]:
# ========== 执行抓取 ==========
# 运行此 cell 来获取所有 16 家公司的 URL

print("=" * 60)
print("开始获取所有 ticker 的 Earnings Call Transcript URL")
print("=" * 60)
print(f"目标公司: {TICKER_LIST}")
print(f"时间范围: {MIN_YEAR}年及以后")
print(f"Chrome 端口: 9222")
print("\n请确保已运行以下命令启动 Chrome:")
print("nohup /Applications/Google\\ Chrome.app/Contents/MacOS/Google\\ Chrome \\")
print("  --remote-debugging-port=9222 \\")
print("  --user-data-dir=/tmp/chrome_sa \\")
print("  >/tmp/chrome_sa.log 2>&1 &")
print("=" * 60)

# 执行抓取
await connect_and_scrape(["CVX"], min_year=MIN_YEAR, check_missing=True)

开始获取所有 ticker 的 Earnings Call Transcript URL
目标公司: ['JPM', 'BK', 'DAL', 'C', 'WFC', 'BAC', 'TSM', 'MS', 'GS', 'BLK', 'MTB', 'STT', 'PNC', 'BR']
时间范围: 2023年及以后
Chrome 端口: 9222

请确保已运行以下命令启动 Chrome:
nohup /Applications/Google\ Chrome.app/Contents/MacOS/Google\ Chrome \
  --remote-debugging-port=9222 \
  --user-data-dir=/tmp/chrome_sa \
  >/tmp/chrome_sa.log 2>&1 &
🚀 正在连接 Chrome（端口 9222）...
✅ 成功连接到 Chrome
📄 将从 1 个未完成的 tickers 开始爬取...

➡️ 正在处理 CVX...
➡️ 导航到 https://seekingalpha.com/symbol/CVX/earnings/transcripts
正在检查是否存在验证码...
未检测到验证码，继续执行。
🔍 检查页面是否包含 transcripts 列表...
📄 使用分页模式抓取所有页面（只保留 Earnings Call Transcript）...
➡️ 抓取第 1 页: https://seekingalpha.com/symbol/CVX/earnings/transcripts?page=1
正在检查是否存在验证码...

检测到验证码！请在浏览器中手动完成验证。
完成后，请回到此终端并按 Enter 键继续。

验证码已解决，程序继续。
📄 第 1 页找到 22 个标题链接（未过滤）
✅ 第 1 页过滤后新增 8 条 Earnings Call Transcript
➡️ 抓取第 2 页: https://seekingalpha.com/symbol/CVX/earnings/transcripts?page=2
正在检查是否存在验证码...
未检测到验证码，继续执行。
📄 第 2 页找到 22 个标题链接（未过滤）
✅ 第 2 页过滤后新增 2 条 Earnings Call Tr